# ValtricAI Research: LoRA Rank Impact on Financial Sentiment Analysis

**Research Question:** How does LoRA rank affect small model performance on financial sentiment?

**Hypothesis:** LoRA with ~1-2% parameters can match full fine-tuning accuracy.

---

## Experiments:
1. **Full Fine-tuning** (baseline) - 100% parameters
2. **LoRA Rank=4** - ~0.8% parameters
3. **LoRA Rank=64** - ~13% parameters

---

**Before running:** Go to `Runtime > Change runtime type` and select **GPU (T4)**

## 1. Setup & Installation

In [ ]:
!pip install -q transformers datasets peft accelerate scikit-learn evaluate

In [ ]:
import os
import time
import json
import warnings
from dataclasses import dataclass, asdict
from typing import Optional

import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, f1_score

warnings.filterwarnings("ignore")

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Configuration

In [ ]:
# Configuration
MODEL_NAME = "distilroberta-base"
MAX_LENGTH = 128
BATCH_SIZE = 32  # Can use larger batch on GPU
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
SEED = 42

# Label mapping
LABEL_MAP = {"bearish": 0, "bullish": 1, "neutral": 2}
ID_TO_LABEL = {0: "bearish", 1: "bullish", 2: "neutral"}

@dataclass
class ExperimentResult:
    """Store results from each experiment run."""
    model: str
    lora_rank: Optional[int]
    params_updated_pct: float
    accuracy: float
    f1_score: float
    training_time_seconds: float
    training_outcome: str
    error_message: Optional[str] = None

## 3. Helper Functions

In [ ]:
def set_seed(seed: int):
    """Set random seeds for reproducibility."""
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def compute_metrics(eval_pred):
    """Compute accuracy and F1 score."""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}

def count_parameters(model):
    """Count total and trainable parameters."""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

## 4. Load Dataset

In [ ]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("Loading dataset...")
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")
dataset = dataset["train"].train_test_split(test_size=0.2, seed=SEED)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

print(f"Train samples: {len(tokenized_dataset['train'])}")
print(f"Test samples: {len(tokenized_dataset['test'])}")

## 5. Experiment 1: Full Fine-Tuning (Baseline)

In [ ]:
print("="*60)
print("EXPERIMENT 1: Full Fine-Tuning (Baseline)")
print("="*60)

set_seed(SEED)
start_time = time.time()

# Load model
model_full = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=ID_TO_LABEL,
    label2id=LABEL_MAP,
)

total_params, trainable_params = count_parameters(model_full)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,} (100%)")

# Training
training_args_full = TrainingArguments(
    output_dir="./results_full_ft",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    seed=SEED,
    report_to="none",
    fp16=torch.cuda.is_available(),  # Use mixed precision on GPU
)

trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

print("\nTraining...")
trainer_full.train()

print("Evaluating...")
eval_full = trainer_full.evaluate()
time_full = time.time() - start_time

result_full = ExperimentResult(
    model="distilroberta-base",
    lora_rank=None,
    params_updated_pct=100.0,
    accuracy=eval_full["eval_accuracy"],
    f1_score=eval_full["eval_f1"],
    training_time_seconds=time_full,
    training_outcome="Success",
)

print(f"\n{'='*40}")
print(f"FULL FINE-TUNING RESULTS")
print(f"{'='*40}")
print(f"Accuracy: {result_full.accuracy:.4f}")
print(f"F1 Score: {result_full.f1_score:.4f}")
print(f"Training Time: {time_full:.1f}s")

# Cleanup
del model_full, trainer_full
torch.cuda.empty_cache()

## 6. Experiment 2: LoRA Rank=4

In [ ]:
print("="*60)
print("EXPERIMENT 2: LoRA Fine-Tuning (Rank = 4)")
print("="*60)

set_seed(SEED)
start_time = time.time()

# Load model
model_lora4 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=ID_TO_LABEL,
    label2id=LABEL_MAP,
)

# Configure LoRA
lora_config_4 = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=4,
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=["query", "value"],
    bias="none",
)

model_lora4 = get_peft_model(model_lora4, lora_config_4)
model_lora4.print_trainable_parameters()

total_params, trainable_params = count_parameters(model_lora4)
original_total = 82120707  # distilroberta-base
params_pct_4 = (trainable_params / original_total) * 100

# Training
training_args_lora4 = TrainingArguments(
    output_dir="./results_lora_r4",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE * 5,  # Higher LR for LoRA
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    seed=SEED,
    report_to="none",
    fp16=torch.cuda.is_available(),
)

trainer_lora4 = Trainer(
    model=model_lora4,
    args=training_args_lora4,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

print("\nTraining...")
trainer_lora4.train()

print("Evaluating...")
eval_lora4 = trainer_lora4.evaluate()
time_lora4 = time.time() - start_time

result_lora4 = ExperimentResult(
    model="distilroberta-base + LoRA",
    lora_rank=4,
    params_updated_pct=round(params_pct_4, 2),
    accuracy=eval_lora4["eval_accuracy"],
    f1_score=eval_lora4["eval_f1"],
    training_time_seconds=time_lora4,
    training_outcome="Success",
)

print(f"\n{'='*40}")
print(f"LORA RANK=4 RESULTS")
print(f"{'='*40}")
print(f"Trainable Params: {params_pct_4:.2f}%")
print(f"Accuracy: {result_lora4.accuracy:.4f}")
print(f"F1 Score: {result_lora4.f1_score:.4f}")
print(f"Training Time: {time_lora4:.1f}s")

# Cleanup
del model_lora4, trainer_lora4
torch.cuda.empty_cache()

## 7. Experiment 3: LoRA Rank=64

In [ ]:
print("="*60)
print("EXPERIMENT 3: LoRA Fine-Tuning (Rank = 64)")
print("="*60)

set_seed(SEED)
start_time = time.time()

# Load model
model_lora64 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=ID_TO_LABEL,
    label2id=LABEL_MAP,
)

# Configure LoRA
lora_config_64 = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=64,
    lora_alpha=128,
    lora_dropout=0.1,
    target_modules=["query", "value"],
    bias="none",
)

model_lora64 = get_peft_model(model_lora64, lora_config_64)
model_lora64.print_trainable_parameters()

total_params, trainable_params = count_parameters(model_lora64)
original_total = 82120707
params_pct_64 = (trainable_params / original_total) * 100

# Training
training_args_lora64 = TrainingArguments(
    output_dir="./results_lora_r64",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE * 5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    seed=SEED,
    report_to="none",
    fp16=torch.cuda.is_available(),
)

trainer_lora64 = Trainer(
    model=model_lora64,
    args=training_args_lora64,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

print("\nTraining...")
trainer_lora64.train()

print("Evaluating...")
eval_lora64 = trainer_lora64.evaluate()
time_lora64 = time.time() - start_time

result_lora64 = ExperimentResult(
    model="distilroberta-base + LoRA",
    lora_rank=64,
    params_updated_pct=round(params_pct_64, 2),
    accuracy=eval_lora64["eval_accuracy"],
    f1_score=eval_lora64["eval_f1"],
    training_time_seconds=time_lora64,
    training_outcome="Success",
)

print(f"\n{'='*40}")
print(f"LORA RANK=64 RESULTS")
print(f"{'='*40}")
print(f"Trainable Params: {params_pct_64:.2f}%")
print(f"Accuracy: {result_lora64.accuracy:.4f}")
print(f"F1 Score: {result_lora64.f1_score:.4f}")
print(f"Training Time: {time_lora64:.1f}s")

# Cleanup
del model_lora64, trainer_lora64
torch.cuda.empty_cache()

## 8. Results Summary

In [ ]:
results = [result_full, result_lora4, result_lora64]

print("\n" + "="*80)
print("VALTRICAI RESEARCH: LORA RANK IMPACT STUDY - FINAL RESULTS")
print("="*80)

print("\n| Configuration | % Params | Accuracy | F1 Score | Time (s) |")
print("|---------------|----------|----------|----------|----------|")

for r in results:
    config = "Full Fine-Tuning" if r.lora_rank is None else f"LoRA (r={r.lora_rank})"
    print(f"| {config:<13} | {r.params_updated_pct:>6.2f}% | {r.accuracy:>8.4f} | {r.f1_score:>8.4f} | {r.training_time_seconds:>8.1f} |")

# Analysis
print("\n" + "="*80)
print("EFFICIENCY ANALYSIS")
print("="*80)

baseline_acc = result_full.accuracy
baseline_time = result_full.training_time_seconds

for r in [result_lora4, result_lora64]:
    acc_diff = (baseline_acc - r.accuracy) * 100
    time_savings = ((baseline_time - r.training_time_seconds) / baseline_time) * 100
    param_savings = 100 - r.params_updated_pct
    
    print(f"\nLoRA Rank={r.lora_rank}:")
    print(f"  - Accuracy difference: {acc_diff:+.2f}% points")
    print(f"  - Training time savings: {time_savings:.1f}%")
    print(f"  - Parameter reduction: {param_savings:.1f}%")

## 9. Save Results

In [ ]:
# Save to JSON
with open("experiment_results.json", "w") as f:
    json.dump([asdict(r) for r in results], f, indent=2)

print("Results saved to experiment_results.json")

# Display JSON
print("\n" + json.dumps([asdict(r) for r in results], indent=2))

## 10. Generate LaTeX Table

In [ ]:
latex_table = r"""
\begin{table}[h]
\centering
\caption{LoRA Rank Impact on Financial Sentiment Classification}
\label{tab:results}
\begin{tabular}{@{}lcccc@{}}
\toprule
\textbf{Configuration} & \textbf{Params (\%)} & \textbf{Accuracy} & \textbf{F1} & \textbf{Time (s)} \\
\midrule
"""

for r in results:
    config = "Full Fine-Tuning" if r.lora_rank is None else f"LoRA ($r={r.lora_rank}$)"
    latex_table += f"{config} & {r.params_updated_pct:.2f} & {r.accuracy:.4f} & {r.f1_score:.4f} & {r.training_time_seconds:.1f} \\\\\n"

latex_table += r"""\bottomrule
\end{tabular}
\end{table}
"""

print("LaTeX Table:")
print(latex_table)

with open("results_table.tex", "w") as f:
    f.write(latex_table)
print("\nSaved to results_table.tex")

---

## Key Findings

After running all experiments, analyze:

1. **Can LoRA match full fine-tuning?**
   - Compare accuracy difference between methods
   
2. **What's the optimal rank?**
   - r=4 vs r=64 tradeoffs
   
3. **Compute savings?**
   - Training time reduction
   - Parameter efficiency gains

---

*ValtricAI Research - December 2025*